In [4]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('AAPL_daily_data.csv')

# Rename "Unnamed: 0" to "Date" and parse it as a datetime column
df.rename(columns={"Unnamed: 0": "Date"}, inplace=True)
df["Date"] = pd.to_datetime(df["Date"], errors="coerce")

# Sort data by date (oldest to newest)
df = df.sort_values(by="Date").reset_index(drop=True)

In [ ]:

import pandas as pd

def calculate_true_range(df):
    """Calculate True Range for ATR."""
    df['High-Low'] = df['high'] - df['low']
    df['High-Close'] = abs(df['high'] - df['close'].shift(1))
    df['Low-Close'] = abs(df['low'] - df['close'].shift(1))
    df['True Range'] = df[['High-Low', 'High-Close', 'Low-Close']].max(axis=1)
    return df
# Volatility signal
def calculate_atr(df, period=14):
    """Calculate Average True Range (ATR) using Wilder's smoothing."""
    df = calculate_true_range(df)
    df['ATR'] = df['True Range'].ewm(alpha=1/period, adjust=False, min_periods=1).mean()
    return df
# Stock trend discovery
def calculate_adx(df, period=14):
    """Calculate Average Directional Index (ADX)."""
    df = calculate_atr(df, period)  # Ensure ATR uses Wilder's smoothing

    # Calculate +DM and -DM
    df['+DM'] = df['high'] - df['high'].shift(1)  # Up move
    df['-DM'] = df['low'].shift(1) - df['low']    # Down move (previous_low - current_low)

    # Set negative DM values to 0
    df['+DM'] = df['+DM'].clip(lower=0)
    df['-DM'] = df['-DM'].clip(lower=0)

    # Zero out the smaller DM
    mask = df['+DM'] > df['-DM']
    df.loc[~mask, '+DM'] = 0
    df.loc[mask, '-DM'] = 0

    # Smooth +DM and -DM using Wilder's method
    df['Smoothed+DM'] = df['+DM'].ewm(alpha=1/period, adjust=False).mean()
    df['Smoothed-DM'] = df['-DM'].ewm(alpha=1/period, adjust=False).mean()

    # Calculate +DI and -DI
    df['+DI'] = (df['Smoothed+DM'] / df['ATR']) * 100
    df['-DI'] = (df['Smoothed-DM'] / df['ATR']) * 100

    # Calculate DX and ADX (smoothed DX)
    df['DX'] = (abs(df['+DI'] - df['-DI']) / (df['+DI'] + df['-DI'])) * 100
    df['ADX'] = df['DX'].ewm(alpha=1/period, adjust=False).mean()

    return df
# Volume weights
def calculate_obv(df):
    """Calculate On-Balance Volume (OBV)."""
    df['OBV'] = (df['volume'] * ((df['close'].diff() > 0) * 2 - 1)).cumsum()
    return df
# Buy&Sell signals
def calculate_rsi(df, period=14):
    """Calculate Relative Strength Index (RSI)."""
    delta = df['close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period, min_periods=1).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period, min_periods=1).mean()
    rs = gain / loss
    df['RSI'] = 100 - (100 / (1 + rs))
    df['RSI'] = df['RSI'].fillna(50)  # Default initial RSI to 50
    return df
# Buy&Sell signals
def calculate_macd(df, short_window=12, long_window=26, signal_window=9):
    """Calculate Moving Average Convergence Divergence (MACD)."""
    df['EMA_12'] = df['close'].ewm(span=short_window, adjust=False, min_periods=1).mean()
    df['EMA_26'] = df['close'].ewm(span=long_window, adjust=False, min_periods=1).mean()
    df['MACD'] = df['EMA_12'] - df['EMA_26']
    df['Signal_Line'] = df['MACD'].ewm(span=signal_window, adjust=False, min_periods=1).mean()
    df['MACD_Histogram'] = df['MACD'] - df['Signal_Line']
    return df

def calculate_technical_indicators(df):
    """Compute technical indicators including ADX, ATR, OBV, RSI, and MACD."""
    df = df.copy()
    df = calculate_atr(df, period=14)
    df = calculate_adx(df, period=14)
    df = calculate_obv(df)
    df = calculate_rsi(df, period=14)
    df = calculate_macd(df, short_window=12, long_window=26, signal_window=9)
    return df


In [6]:
# Apply the technical indicators to the dataset
df = calculate_technical_indicators(df)

# Generate Summary Table
summary_df = df[["Date", "close", "ATR", "ADX", "OBV", "RSI", "MACD", "Signal_Line", "MACD_Histogram"]].tail(10)

# Save results to CSV
df.to_csv("AAPL_technical_analysis.csv", index=False)
summary_df.to_csv("AAPL_summary_table.csv", index=False)

# Display the summary table in a readable format
print(summary_df.to_string(index=False))

      Date  close      ATR   ADX         OBV       RSI      MACD  Signal_Line  MACD_Histogram
2025-02-07 227.63 7.012857 100.0 -2745500508 47.600572 -2.411660    -2.650009        0.238349
2025-02-10 227.65 6.497500 100.0 -2712384863 56.014406 -2.595056    -2.639018        0.043962
2025-02-11 232.62 6.729643 100.0 -2658666501 59.674224 -2.312703    -2.573755        0.261053
2025-02-12 236.87 6.840357 100.0 -2613423209 63.340739 -1.726098    -2.404224        0.678126
2025-02-13 241.53 7.022493 100.0 -2559809155 67.592419 -0.875100    -2.098399        1.223299
2025-02-14 244.60 6.678921 100.0 -2518912928 64.955357  0.046510    -1.669417        1.715927
2025-02-18 244.47 6.179636 100.0 -2567735419 57.571324  0.757669    -1.184000        1.941669
2025-02-19 244.87 5.965679 100.0 -2535531204 56.834532  1.338119    -0.679576        2.017695
2025-02-20 245.83 5.887821 100.0 -2503214297 60.430380  1.854219    -0.172817        2.027036
2025-02-21 245.55 5.153536 100.0 -2556411728 62.503273  2.21